In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import traci
import sys
import argparse
import tensorflow as tf
import numpy as np
from dotenv import load_dotenv

# Import internal modules
from rl_package.rl_logic.Environnement import EnvironnementSumo
from rl_package.rl_logic.Agent import AgentSumo
from rl_package.params import *
print(BATCH_SIZE,NAME_SIMULATION,WINDOW)

32 simu_simple 200


In [3]:


# Load environment variables
load_dotenv()

SUMO_GUI_BIN = '/Library/Frameworks/EclipseSUMO.framework/Versions/1.22.0/EclipseSUMO/bin/sumo-gui'
#SUMO_BIN = r"C:/Program Files/rl_project/Eclipse/Sumo/bin/sumo.exe"
# SIMUL_CONFIG = r"double_traffic/double_traffic.sumo.cfg"
SIMUL_CONFIG
WINDOW=200
BATCH_SIZE=6
# SUMO command

In [4]:
def preprocess():
    """
    Determines the number of inputs and outputs required for each agent.
    """
    sumoCmd = [SUMO_BIN, "-c", SIMUL_CONFIG, '--start', '--no-warnings']
    env = EnvironnementSumo(sumoCmd, WINDOW)
    inputs_per_agents = []
    outputs_per_agents = []
    positions_phases = []

    for trafficlight in env.trafficlights_ids:
        # Get the number of lanes controlled by this traffic light
        n_lanes = len(env.control_lanes(trafficlight))
        inputs_per_agents.append(n_lanes * 3)  # Inputs: queue + vehicle count

        # Get the number of valid traffic light phases (excluding yellow)
        n_phases,position = env.get_phase_without_yellow(trafficlight)
        #print(f'trafficlight :{trafficlight},\n lane associated {env.control_lanes(trafficlight)}')
        n_outputs = len(n_phases)
        positions_phases.append(position)
        outputs_per_agents.append(n_outputs)

    env.close()
    return inputs_per_agents, outputs_per_agents,positions_phases  # List of inputs, outputs per agent, and the postion phases of each trafficlight


In [5]:
def train_models(inputs_per_agents, outputs_per_agents, position_phases, type_model="DQN"):
    """
    Trains multiple reinforcement learning agents to optimize traffic lights.
    Saves each model separately.
    """
    agents = [AgentSumo(type_model, inputs, outputs) for inputs, outputs in zip(inputs_per_agents, outputs_per_agents)]

    # Load pre-trained models if available
    for i, agent in enumerate(agents):
        agent.build_model()
        model_path = f"models/{NAME_SIMULATION}_{type_model}_Agent{i}.keras"
        if os.path.exists(model_path):
            print(f"🔄 Loading pre-trained model for Agent {i} from {model_path}...")
            agent.model_action = tf.keras.models.load_model(model_path)
            if agent.model_target:  # For Double/Dueling DQN
                agent.model_target = tf.keras.models.load_model(model_path)

    sumoCmd = [SUMO_BIN, "-c", SIMUL_CONFIG, '--start', '--no-warnings','--scale','0.4']



    for episode in range(EPISODE):
        print(f'🔄 Episode {episode}/{EPISODE}')
        env = EnvironnementSumo(sumoCmd, WINDOW)

        #Store the position phases of the trafficlight in the environment
        env.position_phases = position_phases

        epsilon = max(1 - episode / EPISODE, 0.01)  # Decaying epsilon for exploration

        traffic_lights = env.trafficlights_ids
        states = [env.get_states_per_traffic_light(traffic_light) for traffic_light in traffic_lights]

        for _ in range(50):  # Steps per episode
            actions = [agent.epsilon_greedy_policy(np.array(states[i]), epsilon) for i, agent in enumerate(agents)]
            next_states, rewards = env.step(actions)

            for i in range(len(agents)):
                agents[i].add_to_memory(np.array(states[i]), np.array(actions[i]), np.array(rewards[i]), np.array(next_states[i]))

            states = next_states

            if len(agents[0].replay_buffer) >= BATCH_SIZE *1:
                for agent in agents:
                    agent.training_step(BATCH_SIZE)

            if env.get_total_number_vehicles() == 0:
                break  # Stop simulation if no vehicles left

        # Update target networks every 5 episodes for Double/Dueling DQN
        if episode % 5 == 0 and type_model != 'DQN':
            for agent in agents:
                agent.model_target.set_weights(agent.model_action.get_weights())

        env.close()

    for i, agent in enumerate(agents):
        model_path = f"models/{NAME_SIMULATION}_{type_model}_Agent{i}.keras"
        agent.model_action.save(model_path)
        print(f"✅ Model saved for Agent {i} at: {model_path}")


In [5]:

def load_trained_agents(inputs_per_agents, outputs_per_agents, type_model="DQN"):
    """
    Loads pre-trained agents from saved model files.
    If any model is missing, exits the program.
    """
    agents = [AgentSumo(type_model, inputs, outputs) for inputs, outputs in zip(inputs_per_agents, outputs_per_agents)]

    for i, agent in enumerate(agents):
        model_path = f"models/{NAME_SIMULATION}_{type_model}_Agent{i}.keras"
        if os.path.exists(model_path):
            print(f"🔄 Loading pre-trained model for Agent {i} from {model_path}...")
            agent.build_model()
            agent.model_action = tf.keras.models.load_model(model_path)
        else:
            print(f"❌ No pre-trained model found for Agent {i}.")
            sys.exit(1)

    return agents

In [6]:
def scenario(agents,positions_phases):
    """
    Runs a SUMO simulation using the trained agents.
    """
    sumoCmd = [SUMO_GUI_BIN, "-c", SIMUL_CONFIG, '--start', '--no-warnings','--scale','1']
    print(sumoCmd)
    WINDOW=200
    env = EnvironnementSumo(sumoCmd, WINDOW)
    #Store the position phases of the trafficlight in the environment
    env.position_phases = positions_phases
    env.full_simul(agents)

In [17]:

type_model = "2DQN"
SIMUL_CONFIG = "/Users/arseneclaustre/code/psels/RL_traffic/Traffic/Traditional_traffic/traditional_traffic.sumo.cfg"
WINDOW=2000
EPISODE = 50
print(NAME_SIMULATION)
inputs_per_agents, outputs_per_agents,positions_phases = preprocess()
print(f'inputs : {inputs_per_agents}')
print(f'outputs : {outputs_per_agents}')
print(f'positions : {positions_phases}')
train_models(inputs_per_agents, outputs_per_agents, positions_phases, type_model)
agents = load_trained_agents(inputs_per_agents, outputs_per_agents, type_model)
scenario(agents,positions_phases)


simu_simple
 Retrying in 1 seconds


Step #0.00 (0ms ?*RT. ?UPS, TraCI: 2ms, vehicles TOT 0 ACT 0 BUF 0)                      
inputs : [36]
outputs : [4]
positions : [[0, 2, 4, 6]]
🚀 Création d'un nouveau modèle 2DQN...
🔄 Loading pre-trained model for Agent 0 from models/simu_simple_2DQN_Agent0.keras...
🔄 Episode 0/50
 Retrying in 1 seconds


Step #1000.00 (1ms ~= 10.00*RT, ~40000.00UPS, TraCI: 53ms, vehicles TOT 324 ACT 40 BUF 0)   ~= 10.00*RT, ~28000.00UPS, TraCI: 52ms, vehicles TOT 235 ACT 28 BUF 5)  
🔄 Episode 1/50
 Retrying in 1 seconds


state [0.   0.   0.01 0.   0.26 0.   0.   0.   0.   0.   0.35 0.   0.   0.
 0.5  0.   1.   0.   0.   0.   0.   0.   1.5  0.   0.   0.   1.   0.
 1.   0.   0.   0.   0.   0.   2.   0.  ]
Q_values [-1.8922786  -0.11480398  1.610095   -0.5270375 ]
Step #720.00 (1ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 234 ACT 19 BUF 0)                  state [0.   0.   0.   0.   0.31 0.   0.   0.   0.   0.   0.4  0.   0.   0.
 0.   0.   1.5  0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   1.5
 3.   0.5  0.   0.   0.   0.   1.   0.  ]
Q_values [-2.4163368  0.2584448  1.8139789 -0.7625415]
Step #1000.00 (0ms ?*RT. ?UPS, TraCI: 54ms, vehicles TOT 324 ACT 31 BUF 0)                 
🔄 Episode 2/50
 Retrying in 1 seconds


state [0.   0.   0.   0.   0.26 0.24 0.   0.   0.   0.   0.94 0.   0.   0.
 0.   0.   1.   1.   0.   0.   0.   0.   2.5  0.   0.   0.   0.   0.
 1.   1.   0.   0.   0.   0.   3.   0.  ]
Q_values [-2.5843313   0.37205225  1.9416676  -0.7460659 ]
Step #1000.00 (0ms ?*RT. ?UPS, TraCI: 54ms, vehicles TOT 324 ACT 34 BUF 0)                  ?*RT. ?UPS, TraCI: 55ms, vehicles TOT 235 ACT 18 BUF 5)                 
🔄 Episode 3/50
 Retrying in 1 seconds


...

state [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Q_values [-0.3142337 -1.6638418  1.482666  -1.7128675]
state [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Q_values [-1.7353607 -3.241036   1.1974955 -4.4434667]
state [0.   0.06 0.03 0.   0.   0.04 0.   0.06 0.04 0.   0.   0.04 0.   1.
 1.   0.   0.   1.   0.   1.   1.   0.   0.   1.   0.   1.5  1.5  0.
 0.   1.   1.   1.   1.   0.5  0.5  1.  ]
Q_values [-0.19308609  0.32991004  1.3970251  -2.9010034 ]
state [0.   0.26 0.26 0.   0.   0.24 0.   0.26 0.24 0.   0.   0.24 0.   1.
 1.5  0.   0.   1.   0.   1.   1.   0.   0.   1.   0.   1.   2.   0.
 0.   1.   0.   1.   1.   0.   0.   1.  ]
Q_values [ 0.3743791  2.2998228  0.7566004 -1.800518 ]
state [0.   0.46 0.   0.   0.   0.44 0.   0.46 0.   0.   0.   0.44 0.   1.
 0.   0.   0.   1.   0.   1.   0.   0.   0.   1.   0.   1.   1.   0.
 0.   1.   1.   1.   0.   

state [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Q_values [-4.504301   -3.0117397  -0.7523184  -0.83519244]
state [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Q_values [-7.920811  -4.9338803 -7.8015833 -6.192869 ]
state [0.   0.06 0.   0.   0.06 0.04 0.   0.06 0.   0.   0.06 0.04 0.   1.
 0.   0.   1.   1.   0.   1.   0.   0.   1.   1.   0.   1.5  0.5  0.
 1.   1.   1.   1.   0.   0.5  1.5  1.  ]
Q_values [-3.8878942  -2.9740868  -0.27328968  0.78223205]
state [0.   0.26 0.01 0.   0.26 0.   0.   0.26 0.   0.   0.29 0.   0.   1.
 0.5  0.   1.   0.   0.   1.   0.   0.   1.5  0.   0.   1.   1.   0.
 1.   0.   0.5  1.   0.   0.   2.   0.  ]
Q_values [-3.0752006 -2.4635425  3.4524646 -3.44132  ]
state [0.   0.66 0.   0.   0.   0.   0.   0.66 0.   0.   0.06 0.   0.   1.
 0.   0.   0.   0.   0.   1.   0.   0.   0.5  0.   0.   1.   1.   1.5
 1.   0.5  1.   1.   

state [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Q_values [-2.3299406  -1.4764807  -0.26594543 -0.20348787]
state [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Q_values [-3.3926077 -5.240678  -7.431978  -4.074883 ]
state [0.   0.   0.03 0.   0.06 0.04 0.   0.   0.04 0.   0.06 0.04 0.   0.
 1.   0.   1.   1.   0.   0.   1.   0.   1.   1.   0.   0.5  1.5  0.
 1.   1.   1.   0.   1.   0.5  1.5  1.  ]
Q_values [-3.0506687  2.8916721  0.8880186 -1.7881856]
state [0.   0.   0.   0.   0.26 0.24 0.   0.   0.   0.   0.28 0.24 0.   0.
 0.   0.   1.   1.   0.   0.   0.   0.   1.5  1.   0.   0.   0.   0.
 1.   1.   0.   0.   0.   0.   2.   1.  ]
Q_values [-1.3753242  -2.3206086   0.74660015  5.9204607 ]
state [0.   0.   0.   0.   0.46 0.   0.   0.   0.   0.   0.67 0.   0.   0.
 0.   0.   1.   0.   0.   0.   0.   0.   2.   0.   0.   0.   1.   0.
 1.   0.   1.   0.   0

state [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Q_values [-0.3228003  -2.6861684   0.42019176  0.69802976]
state [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Q_values [-0.3094094 -3.7665846 -3.725645  -2.7088253]
state [0.   0.   0.03 0.   0.06 0.04 0.   0.   0.04 0.   0.06 0.04 0.   0.
 1.   0.   1.   1.   0.   0.   1.   0.   1.   1.   0.   0.5  1.5  0.
 1.   1.   1.   0.   1.   0.5  1.5  1.  ]
Q_values [-1.3965603  2.2199655  0.3539467 -1.4008619]
state [0.   0.   0.   0.   0.26 0.24 0.   0.   0.   0.   0.28 0.24 0.   0.
 0.   0.   1.   1.   0.   0.   0.   0.   1.5  1.   0.   0.   0.   0.
 1.   1.   0.   0.   0.   0.   2.   1.  ]
Q_values [-0.45226836 -2.3109374   3.5431633  -2.2474005 ]
state [0.   0.   0.   0.   0.   0.44 0.   0.   0.   0.   0.   0.44 0.   0.
 0.   0.   0.   1.   0.   0.   0.   0.   0.   1.   0.   0.   1.   0.
 0.   1.   1.   0.   0

FatalTraCIError: Connection closed by SUMO.

In [ ]:
print(NAME_SIMULATION)

simu_simple


In [7]:
type_model = "2DQN"
SIMUL_CONFIG = "/Users/pr/code/psels/RL_traffic/Traffic/Traditional_traffic/traditional_traffic.sumo.cfg"
inputs_per_agents, outputs_per_agents,positions_phases = preprocess()
agents = load_trained_agents(inputs_per_agents, outputs_per_agents, type_model)

 Retrying in 1 seconds


Step #0.00 (0ms ?*RT. ?UPS, TraCI: 2ms, vehicles TOT 0 ACT 0 BUF 0)                      
🔄 Loading pre-trained model for Agent 0 from models/simu_simple_2DQN_Agent0.keras...
🚀 Création d'un nouveau modèle 2DQN...


In [8]:
print(agents)

In [10]:
scenario(agents,positions_phases)

['/Library/Frameworks/EclipseSUMO.framework/Versions/1.22.0/EclipseSUMO/bin/sumo-gui', '-c', '/Users/pr/code/psels/RL_traffic/Traffic/Traditional_traffic/traditional_traffic.sumo.cfg', '--start', '--no-warnings', '--scale', '1']
 Retrying in 1 seconds


X Fatal error.


Could not connect to TraCI server at localhost:60662 [Errno 61] Connection refused
 Retrying in 1 seconds
Could not connect to TraCI server at localhost:60662 [Errno 61] Connection refused
 Retrying in 1 seconds
Could not connect to TraCI server at localhost:60662 [Errno 61] Connection refused
 Retrying in 1 seconds
Could not connect to TraCI server at localhost:60662 [Errno 61] Connection refused
 Retrying in 1 seconds
200
state [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Q_values [-1.8551202  -2.4557972  -0.83280563 -0.6843622 ]
200
state [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  1.5 2.  0.5 1.  1.  1.  2.  1.  1.  1.5 2.  0.5]
Q_values [-2.7647412 -4.1467075 -7.3522043 -4.015019 ]
200
state [0.   0.   0.01 0.   0.04 0.01 0.   0.   0.01 0.   0.05 0.02 0.   0.
 0.5  0.   1.   0.5  0.   0.   0.5  0.   1.   0.5  0.5  0.5  3.   1.
 1.   1.   2.5  0.   1.   0.5  3.5  1

X Fatal error.
